## Quarterbacks

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

First, we pull 2022 offense data and clean it

In [19]:
df, y = Fantasy.read_data('2021')

Offense_2020
Offense_Proj
Offense_Prior_Actuals
Offense_Proj


In [20]:
# print(y)

           Player  points
0        J. Allen  417.58
1      J. Herbert  395.76
2      P. Mahomes  374.66
3        T. Brady  386.74
4       J. Burrow  328.24
..            ...     ...
370  E. St. Brown   11.20
371     P. Taylor   15.20
372      K. Smith    8.70
373    G. Gilbert    7.76
374      D. Brown   16.10

[375 rows x 2 columns]


In [12]:
# df_year_2, y_2 = Fantasy.read_data('2022')

Split data by positional group

In [4]:
QB_data = football.makeRegularData(df, "QB")


Next, we pull more advanced data for rushing and passing

In [40]:
# Passing advanced
passing_advanced = football.getAdvancedStats("passing", "2021")

Now we are ready to merge the datasets to incorporate the more advance statistics

In [145]:
QB1 = football.addAdvanced(QB_data, passing_advanced)
QB_merged = football.simpleMerge(QB1, y)
QB_final = football.fix_columns_QB(QB_merged)
#There were 2 columns called player
QB_final = QB_final.loc[:,~QB_final.columns.duplicated()]
#Trying to get rid of outliers
QB_final = QB_final[QB_final['GP'] != '-']
QB_final = QB_final[QB_final['GP'] >= 12]
QB_final = QB_final[QB_final['points'] >= 100]
#Splitting up the train and test data
QB_model_X = QB_final.loc[:, ~QB_final.columns.isin(['Player', 'Name', 'Pos', 'Tm', 'points'])]
QB_model_Y = QB_final.loc[:, QB_final.columns.isin(['points'])]


In [146]:
QB_final

,Player,Pos,Tm,GP,Comp,Inc,Passing_Yds,Passing_Td,Int,Pic6,...,Cmp,Yds,IAY,IAY/PA,CAY,CAY/Cmp,CAY/PA,YAC,YAC/Cmp,points
0,L. Jackson,QB,Bal,15,242,134,2757,26,9,1,...,246.0,2882.0,3543.0,9.3,1734.0,7.0,4.5,1148.0,4.7,252.98
1,J. Allen,QB,Buf,16,396,176,4544,37,10,0,...,409.0,4407.0,5304.0,8.2,2664.0,6.5,4.1,1743.0,4.3,417.58
2,R. Wilson,QB,Sea,16,384,174,4212,40,13,1,...,259.0,3113.0,3956.0,9.9,1730.0,6.7,4.3,1383.0,5.3,248.82
3,K. Murray,QB,Ari,16,375,183,3971,26,12,1,...,333.0,3787.0,3847.0,8.0,1986.0,6.0,4.1,1801.0,5.4,310.48
4,P. Mahomes,QB,KC,15,390,198,4740,38,6,0,...,436.0,4839.0,4829.0,7.3,2140.0,4.9,3.3,2699.0,6.2,374.66
6,J. Herbert,QB,LAC,15,396,199,4336,31,10,0,...,443.0,5014.0,5076.0,7.6,2627.0,5.9,3.9,2387.0,5.4,395.76
7,T. Brady,QB,TB,16,401,209,4633,40,12,2,...,485.0,5316.0,5791.0,8.1,2762.0,5.7,3.8,2554.0,5.3,386.74
8,A. Rodgers,QB,GB,16,372,154,4299,48,5,1,...,366.0,4115.0,4099.0,7.7,1947.0,5.3,3.7,2168.0,5.9,336.30
10,R. Tannehill,QB,Ten,16,315,166,3819,33,7,0,...,357.0,3734.0,3959.0,7.5,1958.0,5.5,3.7,1776.0,5.0,282.36
11,M. Stafford,QB,LAR,16,339,189,4084,26,10,3,...,404.0,4886.0,5092.0,8.5,2760.0,6.8,4.6,2126.0,5.3,346.74


In [147]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(QB_model_X, QB_model_Y,
                                   random_state=104, 
                                   test_size=0.5, 
                                   shuffle=True)

In [150]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
r_sq = model.score(X_train, y_train)
print(f"coefficient of determination: {r_sq}")
print(f"intercept: {model.intercept_}")
print(f"slope: {model.coef_}")

coefficient of determination: 1.0
intercept: [35.85644187]
slope: [[-0.11902523  0.31915763 -1.11499423  0.04393625  0.14968442 -0.32324969
  -0.23083102  0.40671704 -0.52550493  0.5387646  -0.05726771 -0.08276165
   0.13043264  0.3928148   0.05342105 -0.08291268 -1.37601774  0.06679482
   0.06679482  0.37299061  0.01838952  0.01341496 -0.04217031  0.01986795
  -0.00691065 -0.02222361 -0.00147843 -0.01187943]]


In [151]:
y_test

,points
25,270.96
35,209.68
16,307.34
33,202.50
15,234.92
2,248.82
12,321.16
29,170.28
27,265.02
31,185.50


In [152]:
model.score(X_test, y_test)

-0.7414152292978844

In [169]:
values = model.coef_
# values
inputs = QB_final.head(1)
inputs = inputs.loc[:, ~inputs.columns.isin(['Player', 'Name', 'Pos', 'Tm', 'points'])]
inputs.columns
combined = (inputs * values)
combined.sum(axis = 1)

GP              -1.785378
Comp            77.236146
Inc           -149.409227
Passing_Yds    121.132238
Passing_Td       3.891795
Int             -2.909247
Pic6            -0.230831
Sks             11.794794
Passing_1st     -72.51968
Att             85.663571
Rushing_Yds    -57.554045
Rushing_Td      -0.579332
Rushing_1st      7.304228
Total            3.928148
Lost             0.213684
Rk              -2.072817
Age            -33.024426
G                0.801538
GS               0.801538
Cmp              91.75569
Yds             52.998593
IAY             47.529191
IAY/PA          -0.392184
CAY             34.451019
CAY/Cmp         -0.048375
CAY/PA          -0.100006
YAC             -1.697235
YAC/Cmp         -0.055833
dtype: object

### CONTRACTS AND OTHER DATA (not yet completed)

We are not using the player contracts yet but here is where we scrape that data

In [7]:
table_contracts = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/offense/all-time/free-agent-2021/limit-100/')

player_contracts = table_contracts[0]
player_contracts

,Rank,Player,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed
0,1,Jay Cutler Quarterback | 2014-2020 (FA: 2021)...,30,7,"$126,700,000","$18,100,000",NaN,"$38,000,000","$54,000,000"
1,2,Cam Newton Quarterback | 2015-2020 (FA: 2021),26,5,"$103,800,000","$20,760,000","$22,500,000","$41,000,000","$60,000,000"
2,3,Andy Dalton Quarterback | 2014-2020 (FA: 2021...,26,6,"$96,000,000","$16,000,000","$12,000,000","$17,000,000","$17,000,000"
3,4,Kirk Cousins Quarterback | 2018-2020 (FA: 2021),29,3,"$84,000,000","$28,000,000","$3,000,000","$84,000,000","$84,000,000"
4,5,Ryan Tannehill Quarterback | 2015-2020 (FA: 2...,26,4,"$77,000,000","$19,250,000","$11,500,000","$21,500,000","$45,000,000"
...,...,...,...,...,...,...,...,...,...
95,96,Paxton Lynch Quarterback | 2016-2020 (FA: 202...,22,4,"$9,476,296","$2,369,074","$5,091,852","$7,884,074","$7,884,074"
96,97,Lee Smith Tight End | 2018-2020 (FA: 2021) O...,30,3,"$9,000,000","$3,000,000",NaN,"$3,000,000","$3,000,000"
97,97,Sammy Watkins Wide Receiver | 2020-2020 (FA: ...,26,1,"$9,000,000","$9,000,000","$1,000,000","$8,000,000","$8,000,000"
98,97,Donte Moncrief Wide Receiver | 2019-2020 (FA:...,25,2,"$9,000,000","$4,500,000","$3,500,000","$3,500,000","$3,500,000"
